\title{Memories in myHDL}
\author{Steven K Armour}
\maketitle

In [1]:
from myhdl import *
from myhdlpeek import Peeker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sympy import *
init_printing()

import random

In [2]:
#helper  functions to read in the .v and .vhd generated files into python
def VerilogTextReader(loc, printresult=True):
    with open(f'{loc}.v', 'r') as vText:
        VerilogText=vText.read()
    if printresult:
        print(f'***Verilog modual from {loc}.v***\n\n', VerilogText)
    return VerilogText

def VHDLTextReader(loc, printresult=True):
    with open(f'{loc}.vhd', 'r') as vText:
        VerilogText=vText.read()
    if printresult:
        print(f'***VHDL modual from {loc}.vhd***\n\n', VerilogText)
    return VerilogText

# Read Only Memory (ROM)
ROM is a memory structure that holds static information that can only be read from. In other words, these are hard-coded instruction memory. That *should* never change. Furthermore, this data is held in a sort of array; for example, we can think of a python tuple as a sort of read-only memory since the content of a tuple is static and we use array indexing to access a certain portions of the memory.

In [3]:
#use type casting on list genrator to store 0-9 in 8bit binary
TupleROM=tuple([bin(i, 8) for i in range(10)])
TupleROM

('00000000',
 '00000001',
 '00000010',
 '00000011',
 '00000100',
 '00000101',
 '00000110',
 '00000111',
 '00001000',
 '00001001')

In [4]:
f'accesss location 6: {TupleROM[6]}, read contents of location 6 to dec:{int(TupleROM[6], 2)}'

'accesss location 6: 00000110, read contents of location 6 to dec:6'

And if we try writing to the tuple we will get an error

In [5]:
#TupleROM[6]=bin(16,2)

# Random and Sequntial Access Memory
So to start off with the Random in RAM does not mean Random in a proplositc sence. It refares to Random as in you can randomly access any part of the data array opposed to the now specility sequantil only memory wich are typicly made with a counter or stat machine to sequance that acation



# HDL Memeorys
in HDL ROM the data is stored a form of a D flip flop that are structerd in a sort of two diminal array where one axis is the address and the other is the content and we use a mux to contorl wich address "row" we are trying to read. There fore we have two signals: address and content. Where the address contorls the mux.

## ROM Preloaded

In [6]:
@block
def ROMLoaded(addr, dout):
    """
    A ROM laoded with data already incoded in the structer
    insted of using myHDL inchanced parmter loading
    
    I/O:
        addr(Signal>4): addres; range is from 0-3
        dout(Signal>4): data at each address
    """
    @always_comb
    def readAction():
        if addr==0:
            dout.next=3
        elif addr==1:
            dout.next=2
        elif addr==2:
            dout.next=1
        
        elif addr==3:
            dout.next=0
    
    return instances()
    

In [7]:
Peeker.clear()
addr=Signal(intbv(0)[4:]); Peeker(addr, 'addr')
dout=Signal(intbv(0)[4:]); Peeker(dout, 'dout')

DUT=ROMLoaded(addr, dout)

def ROMLoaded_TB():
    @instance
    def stimules():
        for i in range(3+1):
            addr.next=i
            yield delay(1)
        raise StopSimulation()
        
    return instances()

sim = Simulation(DUT, ROMLoaded_TB(), *Peeker.instances()).run()

In [8]:
Peeker.to_wavedrom()

In [9]:
Peeker.to_dataframe()

,addr,dout
0,0,3
1,1,2
2,2,1
3,3,0


In [10]:
DUT.convert()
VerilogTextReader('ROMLoaded');

***Verilog modual from ROMLoaded.v***

 // File: ROMLoaded.v
// Generated by MyHDL 0.10
// Date: Thu May 24 20:30:37 2018


`timescale 1ns/10ps

module ROMLoaded (
    addr,
    dout
);
// A ROM laoded with data already incoded in the structer
// insted of using myHDL inchanced parmter loading
// 
// I/O:
//     addr(Signal>4): addres; range is from 0-3
//     dout(Signal>4): data at each address

input [3:0] addr;
output [3:0] dout;
reg [3:0] dout;




always @(addr) begin: ROMLOADED_READACTION
    case (addr)
        'h0: begin
            dout = 3;
        end
        'h1: begin
            dout = 2;
        end
        'h2: begin
            dout = 1;
        end
        'h3: begin
            dout = 0;
        end
    endcase
end

endmodule



With myHDL we can dynamicaly load the contents that will be hard coded in the convertion to verilog/VHDL wich is an ammazing benfict for devlopment as is sean here

In [11]:
@block
def ROMParmLoad(addr, dout, CONTENT):
    """
    A ROM laoded with data from CONTENT input tuple
    
    I/O:
        addr(Signal>4): addres; range is from 0-3
        dout(Signal>4): data at each address
    Parm:
        CONTENT: tuple size 4 with contende must be no larger then 4bit
    """
    @always_comb
    def readAction():
        dout.next=CONTENT[int(addr)]
    
    return instances()
    

In [12]:
Peeker.clear()
addr=Signal(intbv(0)[4:]); Peeker(addr, 'addr')
dout=Signal(intbv(0)[4:]); Peeker(dout, 'dout')
CONTENT=tuple([i for i in range(4)][::-1])

DUT=ROMParmLoad(addr, dout, CONTENT)

def ROMParmLoad_TB():
    @instance
    def stimules():
        for i in range(3+1):
            addr.next=i
            yield delay(1)
        raise StopSimulation()

        
    return instances()

sim = Simulation(DUT, ROMParmLoad_TB(), *Peeker.instances()).run()

In [13]:
Peeker.to_wavedrom()

In [14]:
Peeker.to_dataframe()

,addr,dout
0,0,3
1,1,2
2,2,1
3,3,0


In [15]:
DUT.convert()
VerilogTextReader('ROMParmLoad');

***Verilog modual from ROMParmLoad.v***

 // File: ROMParmLoad.v
// Generated by MyHDL 0.10
// Date: Thu May 24 20:30:38 2018


`timescale 1ns/10ps

module ROMParmLoad (
    addr,
    dout
);
// A ROM laoded with data from CONTENT input tuple
// 
// I/O:
//     addr(Signal>4): addres; range is from 0-3
//     dout(Signal>4): data at each address
// Parm:
//     CONTENT: tuple size 4 with contende must be no larger then 4bit

input [3:0] addr;
output [3:0] dout;
reg [3:0] dout;




always @(addr) begin: ROMPARMLOAD_READACTION
    case (addr)
        0: dout = 3;
        1: dout = 2;
        2: dout = 1;
        default: dout = 0;
    endcase
end

endmodule



we can also create rom that insted of being asynrinues is syncrones 

In [16]:
@block
def ROMParmLoadSync(addr, dout, clk, rst, CONTENT):
    """
    A ROM laoded with data from CONTENT input tuple
    
    I/O:
        addr(Signal>4): addres; range is from 0-3
        dout(Signal>4): data at each address
        clk (bool): clock feed
        rst (bool): reset
    Parm:
        CONTENT: tuple size 4 with contende must be no larger then 4bit
    """
    @always(clk.posedge)
    def readAction():
        if rst:
            dout.next=0
        else:
            dout.next=CONTENT[int(addr)]
    
    return instances()
    

In [17]:
Peeker.clear()
addr=Signal(intbv(0)[4:]); Peeker(addr, 'addr')
dout=Signal(intbv(0)[4:]); Peeker(dout, 'dout')
clk=Signal(bool(0)); Peeker(clk, 'clk')
rst=Signal(bool(0)); Peeker(rst, 'rst')
CONTENT=tuple([i for i in range(4)][::-1])

DUT=ROMParmLoadSync(addr, dout, clk, rst, CONTENT)

def ROMParmLoadSync_TB():
    
    @always(delay(1))
    def ClkGen():
        clk.next=not clk
    
    @instance
    def stimules():
        for i in range(3+1):
            yield clk.posedge
            addr.next=i
        
        for i in range(4):
            yield clk.posedge
            rst.next=1                        
            addr.next=i

        raise StopSimulation()

        
    return instances()

sim = Simulation(DUT, ROMParmLoadSync_TB(), *Peeker.instances()).run()

In [18]:
Peeker.to_wavedrom()

In [19]:
ROMData=Peeker.to_dataframe()
#keep only clock high
ROMData=ROMData[ROMData['clk']==1]
ROMData.drop(columns='clk', inplace=True)
ROMData.reset_index(drop=True, inplace=True)
ROMData

,addr,dout,rst
0,0,3,0
1,1,3,0
2,2,2,0
3,3,1,0
4,0,0,1
5,1,0,1
6,2,0,1


In [20]:
DUT.convert()
VerilogTextReader('ROMParmLoadSync');

***Verilog modual from ROMParmLoadSync.v***

 // File: ROMParmLoadSync.v
// Generated by MyHDL 0.10
// Date: Thu May 24 20:30:38 2018


`timescale 1ns/10ps

module ROMParmLoadSync (
    addr,
    dout,
    clk,
    rst
);
// A ROM laoded with data from CONTENT input tuple
// 
// I/O:
//     addr(Signal>4): addres; range is from 0-3
//     dout(Signal>4): data at each address
//     clk (bool): clock feed
//     rst (bool): reset
// Parm:
//     CONTENT: tuple size 4 with contende must be no larger then 4bit

input [3:0] addr;
output [3:0] dout;
reg [3:0] dout;
input clk;
input rst;




always @(posedge clk) begin: ROMPARMLOADSYNC_READACTION
    if (rst) begin
        dout <= 0;
    end
    else begin
        case (addr)
            0: dout <= 3;
            1: dout <= 2;
            2: dout <= 1;
            default: dout <= 0;
        endcase
    end
end

endmodule



In [26]:
@block
def SeqROMEx(clk, rst, dout):
    """
    Seq Read Only Memory Ex
    I/O:
        clk (bool): clock
        rst (bool): rst on counter
        dout (signal >4): data out
    """
    Count=Signal(intbv(0)[3:])
    
    @always(clk.posedge)
    def counter():
        if rst:
            Count.next=0
        elif Count==3:
            Count.next=0
            
        else:
            Count.next=Count+1
    
    @always(clk.posedge)
    def Memory():
        if Count==0:
            dout.next=3
        elif Count==1:
            dout.next=2
        elif Count==2:
            dout.next=1
        elif Count==3:
            dout.next=0
    
    return instances()

In [27]:
Peeker.clear()
dout=Signal(intbv(0)[4:]); Peeker(dout, 'dout')
clk=Signal(bool(0)); Peeker(clk, 'clk')
rst=Signal(bool(0)); Peeker(rst, 'rst')

DUT=SeqROMEx(clk, rst, dout)

def SeqROMEx_TB():
    
    @always(delay(1))
    def ClkGen():
        clk.next=not clk
    
    @instance
    def stimules():
        for i in range(5+1):
            yield clk.posedge
        
        for i in range(4):
            yield clk.posedge
            rst.next=1                        
            addr.next=i

        raise StopSimulation()

        
    return instances()

sim = Simulation(DUT, SeqROMEx_TB(), *Peeker.instances()).run()

In [28]:
Peeker.to_wavedrom()

In [29]:
SROMData=Peeker.to_dataframe()
#keep only clock high
SROMData=SROMData[SROMData['clk']==1]
SROMData.drop(columns='clk', inplace=True)
SROMData.reset_index(drop=True, inplace=True)
SROMData

,dout,rst
0,3,0
1,2,0
2,1,0
3,0,0
4,3,0
5,2,0
6,1,1
7,0,1
8,3,1


In [30]:
DUT.convert()
VerilogTextReader('SeqROMEx');

***Verilog modual from SeqROMEx.v***

 // File: SeqROMEx.v
// Generated by MyHDL 0.10
// Date: Thu May 24 20:32:52 2018


`timescale 1ns/10ps

module SeqROMEx (
    clk,
    rst,
    dout
);
// Seq Read Only Memory Ex
// I/O:
//     clk (bool): clock
//     rst (bool): rst on counter
//     dout (signal >4): data out

input clk;
input rst;
output [3:0] dout;
reg [3:0] dout;

reg [2:0] Count;



always @(posedge clk) begin: SEQROMEX_COUNTER
    if (rst) begin
        Count <= 0;
    end
    else if ((Count == 3)) begin
        Count <= 0;
    end
    else begin
        Count <= (Count + 1);
    end
end


always @(posedge clk) begin: SEQROMEX_MEMORY
    case (Count)
        'h0: begin
            dout <= 3;
        end
        'h1: begin
            dout <= 2;
        end
        'h2: begin
            dout <= 1;
        end
        'h3: begin
            dout <= 0;
        end
    endcase
end

endmodule



# read and write memory

In [31]:
@block
def RAMConcur(addr, din, writeE, dout, clk):
    """
    Random access read write memeory
    I/O:
        addr(signal>4): the memory cell arrdress
        din (signal>4): data to write into memeory
        writeE (bool): write enable contorl; false is read only
        dout (signal>4): the data out
        clk (bool): clock
        
    Note:
        this is only a 4 byte memory
    """
    #create the memeory list (1D array)
    memory=[Signal(intbv(0)[4:]) for i in range(4)]
    
    @always(clk.posedge)
    def writeAction():
        if writeE:
            memory[addr].next=din
    
    @always_comb
    def readAction():
        dout.next=memory[addr]
    
    return instances()
    

In [32]:
Peeker.clear()
addr=Signal(intbv(0)[4:]); Peeker(addr, 'addr')
din=Signal(intbv(0)[4:]); Peeker(din, 'din')
writeE=Signal(bool(0)); Peeker(writeE, 'writeE')
dout=Signal(intbv(0)[4:]); Peeker(dout, 'dout')
clk=Signal(bool(0)); Peeker(clk, 'clk')
CONTENT=tuple([i for i in range(4)][::-1])

DUT=RAMConcur(addr, din, writeE, dout, clk)

def RAMConcur_TB():
    
    @always(delay(1))
    def ClkGen():
        clk.next=not clk
    
    @instance
    def stimules():
        # do nothing
        for i in range(1):
            yield clk.posedge
        
        #write memory
        for i in range(4):
            yield clk.posedge
            writeE.next=True
            addr.next=i
            din.next=CONTENT[i]
        
        #do nothing
        for i in range(1):
            yield clk.posedge
            writeE.next=False
            
        #read memory
        for i in range(4):
            yield clk.posedge
            addr.next=i

        # rewrite memory
        for i in range(4):
            yield clk.posedge
            writeE.next=True
            addr.next=i
            din.next=CONTENT[-i]
        
        #do nothing
        for i in range(1):
            yield clk.posedge
            writeE.next=False
        
        #read memory
        for i in range(4):
            yield clk.posedge
            addr.next=i
        
        raise StopSimulation()
        
        

        
    return instances()

sim = Simulation(DUT, RAMConcur_TB(), *Peeker.instances()).run()

In [33]:
Peeker.to_wavedrom()

In [34]:
RAMData=Peeker.to_dataframe()
RAMData=RAMData[RAMData['clk']==1]
RAMData.drop(columns='clk', inplace=True)
RAMData.reset_index(drop=True, inplace=True)
RAMData

,addr,din,dout,writeE
0,0,0,0,0
1,0,3,0,1
2,1,2,0,1
3,2,1,0,1
4,3,0,0,1
5,3,0,0,0
6,0,0,3,0
7,1,0,2,0
8,2,0,1,0
9,3,0,0,0


In [35]:
RAMData[RAMData['writeE']==1]

,addr,din,dout,writeE
1,0,3,0,1
2,1,2,0,1
3,2,1,0,1
4,3,0,0,1
10,0,3,3,1
11,1,0,2,1
12,2,1,1,1
13,3,2,0,1


In [36]:
RAMData[RAMData['writeE']==0]

,addr,din,dout,writeE
0,0,0,0,0
5,3,0,0,0
6,0,0,3,0
7,1,0,2,0
8,2,0,1,0
9,3,0,0,0
14,3,2,2,0
15,0,2,3,0
16,1,2,0,0
17,2,2,1,0


In [37]:
DUT.convert()
VerilogTextReader('RAMConcur');

***Verilog modual from RAMConcur.v***

 // File: RAMConcur.v
// Generated by MyHDL 0.10
// Date: Thu May 24 20:33:00 2018


`timescale 1ns/10ps

module RAMConcur (
    addr,
    din,
    writeE,
    dout,
    clk
);
// Random access read write memeory
// I/O:
//     addr(signal>4): the memory cell arrdress
//     din (signal>4): data to write into memeory
//     writeE (bool): write enable contorl; false is read only
//     dout (signal>4): the data out
//     clk (bool): clock
//     
// Note:
//     this is only a 4 byte memory

input [3:0] addr;
input [3:0] din;
input writeE;
output [3:0] dout;
wire [3:0] dout;
input clk;

reg [3:0] memory [0:4-1];



always @(posedge clk) begin: RAMCONCUR_WRITEACTION
    if (writeE) begin
        memory[addr] <= din;
    end
end



assign dout = memory[addr];

endmodule

